In [3]:
import pandas as pd
data=pd.read_csv('house_prices.csv')


In [4]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,N,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,N,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,N,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,N,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,N,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
data['date']=pd.to_datetime(data['date'])

In [6]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,N,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,N,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,N,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,N,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,N,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [7]:
data= data.drop(columns='id', axis=1)

In [8]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [9]:
from catboost import CatBoostRegressor

In [10]:
X=data.drop(columns='price', axis=1)
Y=data['price']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y, test_size=0.2, random_state=42)

In [12]:
model = CatBoostRegressor(verbose=100)

In [13]:
import pickle 
pickle.dump(model, open('save.pkl', 'wb'))

In [15]:
X_pred=model.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mse= mean_squared_error(Y_test, X_pred)

In [18]:
mse

14481691390.788103

In [19]:
import numpy as np
rmse = np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 120339.89941323744


In [20]:
data['price'].max()

7700000.0

In [21]:
from sklearn.model_selection import cross_val_score

In [22]:
X_train_pred=model.predict(X_train)

In [23]:
mse_train= mean_squared_error(Y_train, X_train_pred)
mse_train

4588704516.686749

In [24]:
rmse_train=np.sqrt(mse_train)
rmse_train

67739.97724155766

In [25]:
from sklearn.model_selection import RandomizedSearchCV

In [26]:
para_grid={
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'iterations': [300, 500, 700, 1000],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'bagging_temperature': [0.0, 0.5, 1.0],
    'random_strength': [0.5, 1, 2],
}

search= RandomizedSearchCV(estimator=model, param_distributions=para_grid, n_iter=20, verbose=2, random_state=42)

In [27]:
search.fit(X_train, Y_train, cat_features=cat_features)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0:	learn: 330681.4024134	total: 108ms	remaining: 1m 15s
100:	learn: 92515.3334042	total: 9.22s	remaining: 54.7s
200:	learn: 68326.2357437	total: 19.6s	remaining: 48.6s
300:	learn: 56599.9685714	total: 30.6s	remaining: 40.6s
400:	learn: 48511.8935603	total: 40.9s	remaining: 30.5s
500:	learn: 42203.2532227	total: 50.7s	remaining: 20.1s
600:	learn: 37319.9183194	total: 1m	remaining: 9.98s
699:	learn: 33386.5433509	total: 1m 10s	remaining: 0us
[CV] END bagging_temperature=0.0, depth=10, iterations=700, l2_leaf_reg=3, learning_rate=0.1, random_strength=2; total time= 1.2min
0:	learn: 337323.3211226	total: 95.7ms	remaining: 1m 6s
100:	learn: 89611.7922175	total: 9.49s	remaining: 56.3s
200:	learn: 67592.3718970	total: 19.1s	remaining: 47.5s
300:	learn: 55874.6928789	total: 28.9s	remaining: 38.3s
400:	learn: 48190.7252608	total: 38.7s	remaining: 28.9s
500:	learn: 42459.2336306	total: 48.7s	remaining: 19.3s
600:	learn: 37798.4025839	

RandomizedSearchCV(estimator=<catboost.core.CatBoostRegressor object at 0x00000207728D78C0>,
                   n_iter=20,
                   param_distributions={'bagging_temperature': [0.0, 0.5, 1.0],
                                        'depth': [4, 6, 8, 10],
                                        'iterations': [300, 500, 700, 1000],
                                        'l2_leaf_reg': [1, 3, 5, 7, 9],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'random_strength': [0.5, 1, 2]},
                   random_state=42, verbose=2)

In [28]:
best_model=search.best_estimator_

In [29]:
X_pre_train=best_model.predict(X_train)

In [30]:
mse_train=mean_squared_error( Y_train, X_pre_train)

In [31]:
mse_train

4745014034.691458

In [32]:
rmse_train=np.sqrt(mse_train)

In [33]:
rmse_train

68884.06226908702

In [34]:
X_pre_test=best_model.predict(X_test)
mse_test=mean_squared_error( Y_test, X_pre_test)
rmse_test=np.sqrt(mse_test)
rmse_test


135647.02393787113

In [ ]:
reg.score(X_train, Y_train)

In [37]:
print(data['date'].dtype)

datetime64[ns]


In [38]:
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2014-10-13,221900.0,3,1.00,1180,5650,1.0,N,0,Average,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,2014-12-09,538000.0,3,2.25,2570,7242,2.0,N,0,Average,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2015-02-25,180000.0,2,1.00,770,10000,1.0,N,0,Average,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2014-12-09,604000.0,4,3.00,1960,5000,1.0,N,0,Very Good,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,2015-02-18,510000.0,3,2.00,1680,8080,1.0,N,0,Average,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [39]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

In [40]:
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
0,2014-10-13,221900.0,3,1.00,1180,5650,1.0,N,0,Average,...,1955,0,98178,47.5112,-122.257,1340,5650,2014,10,13
1,2014-12-09,538000.0,3,2.25,2570,7242,2.0,N,0,Average,...,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12,9
2,2015-02-25,180000.0,2,1.00,770,10000,1.0,N,0,Average,...,1933,0,98028,47.7379,-122.233,2720,8062,2015,2,25
3,2014-12-09,604000.0,4,3.00,1960,5000,1.0,N,0,Very Good,...,1965,0,98136,47.5208,-122.393,1360,5000,2014,12,9
4,2015-02-18,510000.0,3,2.00,1680,8080,1.0,N,0,Average,...,1987,0,98074,47.6168,-122.045,1800,7503,2015,2,18
